In [1]:
import snake, queue, random, threading, math, time
import tensorflow as tf
import numpy as np
import tkinter as tk

In [2]:
stack_size = 4
epsilon = 0.1
discount = 0.95
learning_rate = 0.1
batch_size = 10

In [3]:
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

# Q-Network
q = tf.keras.Sequential()

input_1 = (stack_size, 15, 15, 1)
input_2 = (stack_size, 5, 5, 1)

q.add(tf.keras.layers.Conv2D(15, 4,
                             activation="relu", input_shape=input_1[1:]))
q.add(tf.keras.layers.Conv2D(5, 2,
                             activation="relu", input_shape=input_2[1:]))
q.add(tf.keras.layers.Flatten())
q.add(tf.keras.layers.Dense(32, activation="relu"))
q.add(tf.keras.layers.Dense(1, activation="relu"))
q.compile(optimizer="SGD", loss="mse")

q.summary()

# Replay Memory
replay_memory = []

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 12, 12, 15)        255       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 5)         305       
_________________________________________________________________
flatten (Flatten)            (None, 605)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                19392     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 19,985
Trainable params: 19,985
Non-trainable params: 0
_________________________________________________________________


In [4]:
class experience:
    states = None
    action = None
    reward = None
    transitions = None
    
    def __init__(self, states, action, reward, transitions):
        self.states = states
        self.action = action
        self.reward = reward
        self.transitions = transitions

In [5]:
class agent:
    directions = ["UP", "DOWN", "LEFT", "RIGHT"]
    phi = queue.deque()

    def __init__(self, game, return_queue, replay_memory, q, optimizer, epsilon, discount, rate, batch_size):
        self.game = game
        self.return_queue = return_queue
        self.replay_memory = replay_memory
        self.q = q
        self.optimizer = optimizer

        self.epsilon = epsilon
        self.discount = discount
        self.rate = rate
        self.batch_size = batch_size

    def epsilon_action(self):
        if random.randrange(0, 1) <= self.epsilon:
            action = self.directions[random.randint(0, 3)]
        else:
            action = self.directions[np.argmax(q.predict(self.phi))]
        return action

    def step(self):
        action = self.epsilon_action()
        self.game.step(action)

        while self.return_queue.empty():
            if not self.game.running:
                break

        state_reward = return_queue.get()

        phi_last = np.array(self.phi)
        self.phi.appendleft(np.expand_dims(state_reward[0], axis=2))

        phi_current = np.array(self.phi)

        if len(self.phi) > stack_size:
            self.phi.pop()
            self.replay_memory.append(experience(
                phi_last, action, state_reward[1], phi_current))

    def get_batch(self):
        batch = np.empty([self.batch_size], dtype=experience)
        if len(replay_memory) != 0:
            for x in range(self.batch_size):
                batch[x] = replay_memory[random.randint(0, len(replay_memory) - 1)]

        return batch

    def loss(self, e):
        q_phi = self.q(e.states)
        q_phi_next = self.q(e.transitions)
        yj = e.reward + self.discount * np.amax(q_phi_next)
        return math.pow(yj - q_phi[self.directions.index(e.action)], 2)

    def learn(self):
        # This probably isn't correct but might as well try it
        if len(replay_memory) != 0:
            batch = self.get_batch()
            for e in batch:
                optimizer.apply_gradients(optimizer.get_gradients(self.loss(e)))


In [6]:
def train_agent(agent, epoch):
    time.sleep(1)
    print("Training epoch " + str(epoch) + ".")

    while agent.game.running:
        agent.step()
        agent.learn()

    print("Training ended, agent scored " + str(game.score) + " points.")

In [7]:
game = snake.game(queue.Queue(1))
for x in range(100):
    return_queue = queue.Queue(1)

    dqn = agent(game, return_queue, replay_memory, q, optimizer, epsilon, discount, learning_rate, batch_size)

    training_thread = threading.Thread(target=train_agent, args=(dqn, x))
    training_thread.start()

    game.start(return_queue)
    training_thread.join()

game.w.destroy()

Training epoch 0.
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/jason/anaconda3/envs/MLSnake/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/jason/anaconda3/envs/MLSnake/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-9b0dd7c6819d>", line 7, in train_agent
  File "<ipython-input-5-90d790257db2>", line 63, in learn
TypeError: get_gradients() missing 1 required positional argument: 'params'
